# literature review

In [1]:
# ! cd ~/git/openai; pip3 install -e .
# !pip3 install --upgrade openai

In [2]:
import pandas as pd
import textwrap
from tqdm import tqdm
from abcli.modules import objects
from abcli.file import add_postfix
from openai_commands import NAME, VERSION
from openai_commands.completion.api import complete_prompt
from openai_commands.logger import logger

logger.info(f"using {NAME}-{VERSION}.")

🛠️  using openai_commands-3.25.1.


In [3]:
object_name = "AMR-v1"
#filename = "review_463333_screen_csv_20240730130035.csv"
filename = "review_463333_irrelevant_csv_20240730131027.csv"

do_download = False

In [4]:
df = pd.read_csv(objects.path_of(filename, object_name))
logger.info("loaded {:,} item(s).".format(len(df)))

# df = df.head(5)

🛠️  loaded 176 item(s).


In [5]:
def generate_prompt(abstract:str) -> str:
    prompt =  f"""
        read this abstract of a scentific paper and return "antimicrobial" if it talks about antimicrobial 
        treatment for cholera. return "susceptibility" if it has information regarding antibiotic-reistance
        or susceptibility to cholerae strains. otherwise, return "not relevant". 

        {abstract}
        """

    prompt = prompt.replace("\n"," ")
    while "  " in prompt:
        prompt = prompt.replace("  ", " ")

    return prompt

In [6]:
def assess_abstract(abstract: str) -> str:
    prompt = generate_prompt(abstract)
    success, assessment, _ = complete_prompt(prompt)
    assert success

    # logger.info("{}\n---".format("\n".join(textwrap.wrap(f"{abstract}: {assessment}", width=80))))

    return assessment

In [7]:
df["Screening Results"] = df["Abstract"].apply(assess_abstract)

httpx HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
🛠️  response: ChatCompletion(id='chatcmpl-9qXsWhe66fU6fxrfvoDbLDbtgoROd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='not relevant', role='assistant', function_call=None, tool_calls=None))], created=1722310312, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=403, total_tokens=405))
httpx HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
🛠️  response: ChatCompletion(id='chatcmpl-9qXsWhwV1wbqRZB0ytyNRv9vkXL1S', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='antimicrobial', role='assistant', function_call=None, tool_calls=None))], created=1722310312, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=Completi

In [8]:
df.to_csv(objects.path_of(add_postfix(filename, "screening-results"), object_name), index=False)

In [9]:
# END